In [ ]:
from datetime import datetime
import requests
from io import BytesIO
from PIL import Image

import pandas as pd
import numpy as np

import nfl_data_py as nfl

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import plotly.colors as cl

from resources.plotly_theme import nfl_template
from resources.heat_map import heat_map
from resources.get_nfl_data import get_pbp_data, get_team_data
from notebooks.resources.team_stats import get_team_offense_data


pio.templates['nfl_template'] = nfl_template

In [2]:
''' Parameters '''

WEEK = 8
AWAY_TEAM = 'WAS'
HOME_TEAM = 'KC'


# Process

In [ ]:
''' Import Data '''

# pbp_data: pd.DataFrame = nfl.import_pbp_data([2025], downcast=True)

# pbp_data = pbp_data.loc[pbp_data['season_type'] == 'REG', :]

# ## Success
# pbp_data['% ydstogo'] = pbp_data['yards_gained'] / pbp_data['ydstogo']

# pbp_data['Successful Play'] = (
#     ((pbp_data['down'] == 1) & (pbp_data['% ydstogo'] >= 0.4)) |
#     ((pbp_data['down'] == 2) & (pbp_data['% ydstogo'] >= 0.6)) |
#     (pbp_data['first_down'] == 1) |
#     (pbp_data['touchdown'] == 1)
# )

# ## Snaps
# pbp_data['Snap'] = ((pbp_data['pass'] == 1) | (pbp_data['rush']) == 1 | (pbp_data['special'] == 1))

# ## Yard Thresholds
# def distance_range(down, yds):
       
#     down_s = ''
#     match down:
#         case 1:
#             # down_s = '1st'
#             return '1st'
#         case 2:
#             down_s = '2nd'
#         case 3:
#             down_s = '3rd'
#         case 4:
#             down_s = '4th'
#         case default:
#             return ''
        
#     yds_range = ''
#     if yds <= 2:
#         yds_range = 'Short'
#     elif yds <= 6:
#         yds_range = 'Medium'
#     else:
#         yds_range = 'Long'

#     return f'{down_s} & {yds_range}'
    
# pbp_data['Down & Distance'] = pbp_data.apply(lambda x: distance_range(x['down'], x['ydstogo']), axis=1)

2025 done.
Downcasting floats.


In [3]:
''' Import Data '''

# Import
team_data = get_team_data()
pbp_data = get_pbp_data(years=[2025])

player_info = nfl.import_players()

game_data = pbp_data.loc[(pbp_data['week'] == WEEK) & 
                         (pbp_data['home_team'] == HOME_TEAM), :]

print(game_data.shape)

2025 done.
Downcasting floats.


/Users/jmiller/Documents/Fun/nfl/notebooks/resources/get_nfl_data.py:102: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pbp_data['Non-Play Type'] = conditions
/Users/jmiller/Documents/Fun/nfl/notebooks/resources/get_nfl_data.py:105: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  pbp_data['Offensive Snap'] = (((pbp_data['pass'] == 1) | (pbp_data['rush'] == 1)) & (~pbp_data['epa'].isna()))
/Users/jmiller/Documents/Fun/nfl/notebooks/resources/get_nfl_data.py:109: PerformanceWarning: DataFrame is highly fragmented.  This is usually t

(148, 378)


In [92]:
print(player_info.loc[player_info['gsis_id'] == '00-0036223',:].head().to_string())

          gsis_id     display_name common_first_name first_name last_name short_name football_name suffix     esb_id nfl_id    pfr_id pff_id otc_id  espn_id                              smart_id  birth_date position_group position ngs_position_group ngs_position  height  weight                                                                           headshot college_name  college_conference jersey_number  rookie_season  last_season latest_team status ngs_status ngs_status_short_description  years_of_experience pff_position pff_status  draft_year  draft_round  draft_pick draft_team
21068  00-0036223  Jonathan Taylor          Jonathan   Jonathan    Taylor   J.Taylor      Jonathan   None  TAY431618  52449  TaylJo02  57488   8781  4242335  32005441-5943-1618-c081-5dd6b2d0b829  1999-01-19             RB       RB                 RB           RB    70.0   226.0  https://static.www.nfl.com/image/upload/f_auto,q_auto/league/uwmnrhwtwug0wrgkomjk    Wisconsin  Big Ten Conference            28   

# Offense

In [4]:
team_offense = get_team_offense_data(game_data)

print(team_offense.to_string())

         Games  Plays  TDs  FirstDowns  RushAttempts  RushYards  RushTDs  Rush1Ds  DesignedRushPlays  DesignedRushAttempts  DesignedRushYards  QBScrambles  ScrambleYards  DesignedPassPlays  Dropbacks  PassCompletions  PassAttempts  PassYards  PassTDs  Pass1Ds  Sacks  SackYards  INTs  Fumbles  Penalties  PenaltyYards  Penalty1Ds  Drives  Total Yards  Turnovers  PlaysAdv  PassPlays  RushPlays        EPA   RushEPA    PassEPA  Successes  RushSuccesses  PassSuccesses  EPA / Play  Rush EPA / Play  Pass EPA / Play  Success Rate  Rush Success Rate  Pass Success Rate
posteam                                                                                                                                                                                                                                                                                                                                                                                                                                            

## Rushing

In [8]:
''' Rushing '''

run_data = game_data.loc[game_data['play_type'] == 'run', :]
# run_data = pbp_data.loc[pbp_data['play_type'] == 'run', :]


In [9]:
''' Team Rushing '''

by_team = run_data.groupby(['posteam', 'game_half']).aggregate(
    Attempts=('rusher_player_id', 'size'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('Successful Play', 'sum')
)
totals = by_team.groupby(['posteam']).sum()
totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
by_team = pd.concat([by_team, totals]).sort_index()

by_team['Yds / Att'] = round(by_team['Yards'] / by_team['Attempts'], 2)
by_team['Success Rate'] = round((by_team['Successes'] / by_team['Attempts']) * 100, 2)
by_team['1D Rate'] = round((by_team['FirstDowns'] / by_team['Attempts']) * 100, 2)
by_team['TD Rate'] = round((by_team['TDs'] / by_team['Attempts']) * 100, 2)

print(by_team.to_string())


                   Attempts  Yards  TDs  FirstDowns  Successes  Yds / Att  Success Rate  1D Rate  TD Rate
posteam game_half                                                                                        
IND     Half1             9   75.0  1.0         5.0          6       8.33         66.67    55.56    11.11
        Half2             9   89.0  1.0         2.0          2       9.89         22.22    22.22    11.11
        Total            18  164.0  2.0         7.0          8       9.11         44.44    38.89    11.11
TEN     Half1            11   71.0  0.0         2.0          5       6.45         45.45    18.18     0.00
        Half2            14   42.0  1.0         3.0          6       3.00         42.86    21.43     7.14
        Total            25  113.0  1.0         5.0         11       4.52         44.00    20.00     4.00


In [ ]:
''' Player Rushing '''

by_player = run_data.groupby(['posteam', 'rusher_player_name']).aggregate(
    player_id=('rusher_player_id', 'first'),
    Attempts=('rusher_player_id', 'size'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('Successful Play', 'sum')
).sort_values(by=['posteam', 'Attempts'], ascending=False)

by_player['Yds / Att'] = round(by_player['Yards'] / by_player['Attempts'], 2)
by_player['Success Rate'] = round((by_player['Successes'] / by_player['Attempts']) * 100, 2)
by_player['1D Rate'] = round((by_player['FirstDowns'] / by_player['Attempts']) * 100, 2)
by_player['TD Rate'] = round((by_player['TDs'] / by_player['Attempts']) * 100, 2)

print(by_player.to_string())

                             player_id  Attempts  Yards  TDs  FirstDowns  Successes  Yds / Att  Success Rate  1D Rate  TD Rate                                                                           Headshot
posteam rusher_player_name                                                                                                                                                                                       
TEN     T.Pollard           00-0035261        11   44.0  0.0         3.0          5       4.00         45.45    27.27     0.00  https://static.www.nfl.com/image/upload/f_auto,q_auto/league/dvpdczetkdhrqpm8izmd
        T.Spears            00-0039032         9   59.0  1.0         2.0          4       6.56         44.44    22.22    11.11  https://static.www.nfl.com/image/upload/f_auto,q_auto/league/fc7vhbtw80einad9hsfu
        C.Ward              00-0040676         2    4.0  0.0         0.0          1       2.00         50.00     0.00     0.00  https://static.www.nfl.com/image

In [7]:
# Define your colorscale (using a built-in sequential scale as an example)
colorscale_name = 'Greens'
colorscale = cl.sequential.__dict__[colorscale_name]
print(colorscale)

# Extract colors from the colorscale
rgb_colors = cl.colorscale_to_colors(['rgb(0,68,27)'])
print(rgb_colors)

# Convert RGB colors to hex format
hex_colors = cl.convert_colors_to_same_type(rgb_colors, colortype='hex')

print(f"Hex colors from '{colorscale_name}' colorscale:")
for hex_code in hex_colors:
    print(hex_code)

['rgb(247,252,245)', 'rgb(229,245,224)', 'rgb(199,233,192)', 'rgb(161,217,155)', 'rgb(116,196,118)', 'rgb(65,171,93)', 'rgb(35,139,69)', 'rgb(0,109,44)', 'rgb(0,68,27)']
['g']


PlotlyError: You must select either rgb or tuple for your colortype variable.

In [71]:
''' Rush by Down / Distance '''

MIN_ATTEMPTS = 2

## Group by down/distance
rusher_carries = run_data.groupby(['rusher_player_name', 'Down & Distance']).aggregate(
    Attempts=('rusher_player_name', 'size'),
    Successes=('Successful Play', 'sum'),
    player_id=('rusher_player_id', 'first'),
).reset_index()
rusher_carries['Success Rate'] = rusher_carries['Successes'] / rusher_carries['Attempts']

## Pivot
DOWNS = ['1st', '2nd & Short', '2nd & Medium', '2nd & Long', 
       '3rd & Short', '3rd & Medium', '3rd & Long', '4th & Short', '4th & Medium']
col_sort = pd.MultiIndex.from_product([DOWNS, ['Successes', 'Attempts', 'Success Rate']])

piv = rusher_carries.loc[(rusher_carries['Down & Distance'] != ''),:].pivot(
        index='rusher_player_name',
        columns=['Down & Distance'],
        values=['Attempts', 'Successes', 'Success Rate']
    )\
    .swaplevel(0, 1, axis=1).sort_index(axis=1).reset_index().set_index('rusher_player_name')\
    .reindex(columns=col_sort)

print(piv.to_string())

## Visualize Success Rate

# Filter to success rate
idx = pd.IndexSlice
sl = pd.DataFrame(piv.loc[:, idx[:, ('Success Rate')]])
sl.columns = sl.columns.get_level_values(0)

# Merge in total attempts
sl = sl.merge(
        by_player.reset_index().set_index('rusher_player_name')['Attempts'], 
        right_index=True,
        left_index=True,
        how='left'
    ).sort_values(by='Attempts', ascending=False)\
    .reset_index().set_index('rusher_player_name')
# piv = piv.set_index('rusher_player_name')
print(sl.to_string())

# Create annotations
annot_df = sl.copy() # Initialize with data, convert to string
for r in annot_df.index:
    print(r)
    for c in piv.columns.get_level_values(0).unique():
        attempts = piv.loc[r, (c, 'Attempts')]
        if pd.isna(attempts):
            annot_df.loc[r,c] = ''
        else:
            annot_df.loc[r, c] = f"{piv.loc[r, (c, 'Success Rate')]*100:,.1f}%<br>({piv.loc[r,(c, 'Successes')]:,.0f} / {attempts:,.0f})"

# print(annot_df.head().to_string())
# print(sl.head().to_string())
# print(sl.index)

# Heatmap

# fig = go.Figure()

cols = [col.replace(' ', '<br>') for col in sl.columns.get_level_values(0).drop('Attempts')]

fig = heat_map(
    x=cols,
    y=sl.index,
    z=sl.values.tolist(),
    text=annot_df.values.tolist(),
    title=f'Rushing <span style="color: #006D2C">Success Rate</span> by Down & Distance<br><sup>NFL Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>',
    color_name='Success Rate'
)
fig.update_layout(
    margin=dict(t=100, l=100, r=100, b=50),
    width=900,
    height=500
)
fig.update_annotations(y=-.1)

# headshots = rusher_carries['Headshot'].tolist()
# n_players = len(headshots)
# for i in range(n_players):
#     response = requests.get(headshots[i])
#     headshot = Image.open(BytesIO(response.content))
#     fig.add_layout_image(
#         x=-.135,
#         # y=(1-(1/n_players))-((1/n_players)*i),
#         y=1.01-((1/n_players)*i),
#         sizex=1/n_players,
#         sizey=1/n_players,
#         xanchor='center',
#         yanchor='top',
#         xref='paper', 
#         yref='paper',
#         source=headshot,
#     )

fig.show()

# pio.write_image(fig, "Success Rate by Down & Distance - Colts vs. Rams.png", height=500, width=900, scale=6)

                         1st                       2nd & Short                       2nd & Medium                       2nd & Long                       3rd & Short                       3rd & Medium                       3rd & Long                       4th & Short                       4th & Medium                      
                   Successes Attempts Success Rate   Successes Attempts Success Rate    Successes Attempts Success Rate  Successes Attempts Success Rate   Successes Attempts Success Rate    Successes Attempts Success Rate  Successes Attempts Success Rate   Successes Attempts Success Rate    Successes Attempts Success Rate
rusher_player_name                                                                                                                                                                                                                                                                                                                 
A.Abdullah               0.0

## Passing

In [81]:
''' Passing '''

pass_data = game_data.loc[game_data['play_type_nfl'] == 'PASS', :]
# pass_data = pbp_data.loc[(pbp_data['play_type'] == 'pass') | (pbp_data['play_type'] == 'qb_spike'), :]

In [82]:
''' Team Passing '''
# NOTE - Team passing includes sacks, both attempts and yards

by_team = pass_data.groupby(['posteam', 'game_half']).aggregate(
    Attempts=('pass_attempt', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('Successful Play', 'sum'),
    Sacks=('sack', 'sum')
)
totals = by_team.groupby(['posteam']).sum()
totals.index = pd.MultiIndex.from_tuples([(i, 'Total') for i in totals.index])
by_team = pd.concat([by_team, totals]).sort_index()

by_team['Yds / Att'] = round(by_team['Yards'] / by_team['Attempts'], 2)
by_team['Success Rate'] = round((by_team['Successes'] / by_team['Attempts']) * 100, 2)
by_team['1D Rate'] = round((by_team['FirstDowns'] / by_team['Attempts']) * 100, 2)
by_team['TD Rate'] = round((by_team['TDs'] / by_team['Attempts']) * 100, 2)

print(by_team.to_string())

                   Attempts  Yards  TDs  FirstDowns  Successes  Sacks  Yds / Att  Success Rate    1D Rate  TD Rate
posteam game_half                                                                                                 
IND     Half1          19.0  146.0  1.0         9.0         10    0.0       7.68         52.63  47.369999     5.26
        Half2          12.0  126.0  2.0         5.0          6    0.0      10.50         50.00  41.669998    16.67
        Total          31.0  272.0  3.0        14.0         16    0.0       8.77         51.61  45.160000     9.68
TEN     Half1          12.0  109.0  1.0         6.0          6    0.0       9.08         50.00  50.000000     8.33
        Half2          24.0  150.0  0.0         7.0         12    0.0       6.25         50.00  29.170000     0.00
        Total          36.0  259.0  1.0        13.0         18    0.0       7.19         50.00  36.110001     2.78


In [83]:
''' Passing - QBs '''
# NOTE - Player passing does NOT include sacks, but success rate denominator is Attempts + Sacks

# Group by
by_player = pass_data.loc[pass_data['sack'] != 1,:].groupby(['posteam', 'passer_player_name']).aggregate(
    Completions=('complete_pass', 'sum'),
    Attempts=('pass_attempt', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    Ints=('interception', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('Successful Play', 'sum')
)
by_player = by_player.sort_values(by=['posteam', 'Attempts'], ascending=False)

# Add sack data
player_sacks = pass_data.loc[pass_data['sack'] == 1, :].groupby(['posteam', 'passer_player_name']).aggregate(Sacks=('sack', 'sum'), SackYards=('yards_gained', 'sum'))
player_sacks['SackYards'] = abs(player_sacks['SackYards'])
by_player = by_player.merge(player_sacks, left_index=True, right_index=True, how='left').fillna(0)

by_player['Yds / Att'] = round(by_player['Yards'] / by_player['Attempts'], 2)
by_player['Success Rate'] = round((by_player['Successes'] / (by_player['Attempts'] + by_player['Sacks'])) * 100, 2)
by_player['1D Rate'] = round((by_player['FirstDowns'] / by_player['Attempts']) * 100, 2)
by_player['TD Rate'] = round((by_player['TDs'] / by_player['Attempts']) * 100, 2)

print(by_player.to_string())

                            Completions  Attempts  Yards  TDs  Ints  FirstDowns  Successes  Sacks  SackYards  Yds / Att  Success Rate    1D Rate  TD Rate
posteam passer_player_name                                                                                                                               
TEN     C.Ward                     22.0      36.0  259.0  1.0   0.0        13.0         18    0.0        0.0       7.19         50.00  36.110001     2.78
IND     D.Jones                    21.0      29.0  272.0  3.0   0.0        14.0         16    0.0        0.0       9.38         55.17  48.279999    10.34
        R.Leonard                   0.0       2.0    0.0  0.0   0.0         0.0          0    0.0        0.0       0.00          0.00   0.000000     0.00


## Receiving

In [84]:
''' Receiving '''
# NOTE - 

by_player = pass_data.groupby(['posteam', 'receiver_player_name']).aggregate(
    Targets=('receiver_player_name', 'size'),
    Receptions=('complete_pass', 'sum'),
    Yards=('yards_gained', 'sum'),
    TDs=('touchdown', 'sum'),
    FirstDowns=('first_down', 'sum'),
    Successes=('Successful Play', 'sum')
).sort_values(by=['posteam', 'Targets'], ascending=False)

by_player['Yds / Rec'] = round(by_player['Yards'] / by_player['Receptions'], 2)
by_player['Success Rate'] = round((by_player['Successes'] / by_player['Targets']) * 100, 2)
by_player['1D Rate'] = round((by_player['FirstDowns'] / by_player['Targets']) * 100, 2)
by_player['TD Rate'] = round((by_player['TDs'] / by_player['Targets']) * 100, 2)

print(by_player.to_string())

                              Targets  Receptions  Yards  TDs  FirstDowns  Successes  Yds / Rec  Success Rate  1D Rate  TD Rate
posteam receiver_player_name                                                                                                   
TEN     E.Ayomanor                  8         4.0   52.0  0.0         2.0          3      13.00         37.50    25.00     0.00
        V.Jefferson                 8         1.0    6.0  0.0         0.0          1       6.00         12.50     0.00     0.00
        C.Dike                      7         7.0   93.0  0.0         5.0          6      13.29         85.71    71.43     0.00
        C.Okonkwo                   5         4.0   53.0  0.0         2.0          4      13.25         80.00    40.00     0.00
        G.Helm                      3         2.0   23.0  1.0         2.0          2      11.50         66.67    66.67    33.33
        T.Spears                    3         3.0   23.0  0.0         1.0          1       7.67         

In [85]:
''' Receiving by Down / Distance '''

DOWN_COL = 'Down & Distance' #'down'

## Group by down/distance
receiver_targets = pass_data.groupby(['receiver_player_name', DOWN_COL]).aggregate(
    Targets=('receiver_player_name', 'size'),
    Successes=('Successful Play', 'sum')
).reset_index()
receiver_targets['Success Rate'] = receiver_targets['Successes'] / receiver_targets['Targets']

# receiver_targets = receiver_targets.loc[receiver_targets['Targets'] > 1, :]

## Pivot
DOWNS = [1,2,3,4]
DOWNS_SUBGROUP = ['1st', '2nd & Short', '2nd & Medium', '2nd & Long', 
       '3rd & Short', '3rd & Medium', '3rd & Long', '4th & Short', '4th & Medium']
col_sort = pd.MultiIndex.from_product([DOWNS_SUBGROUP, ['Successes', 'Targets', 'Success Rate']])

piv = receiver_targets.loc[(receiver_targets[DOWN_COL] != ''),:].pivot(
        index='receiver_player_name',
        columns=[DOWN_COL],
        values=['Targets', 'Successes', 'Success Rate']
    )\
    .swaplevel(0, 1, axis=1).sort_index(axis=1).reset_index().set_index('receiver_player_name')\
    .reindex(columns=col_sort)

print(piv.to_string())

## Visualize Success Rate

# Filter to success rate
idx = pd.IndexSlice
sl = pd.DataFrame(piv.loc[:, idx[:, ('Success Rate')]])
sl.columns = sl.columns.get_level_values(0)

# Merge in total Targets
sl = sl.merge(
        by_player.reset_index().set_index('receiver_player_name')['Targets'], 
        right_index=True,
        left_index=True,
        how='left'
    ).sort_values(by='Targets', ascending=False)\
    .reset_index().set_index('receiver_player_name')
# piv = piv.set_index('receiver_player_name')
# print(piv.to_string())

# Create annotations
annot_df = sl.copy() # Initialize with data, convert to string
for r in annot_df.index:
    for c in piv.columns.get_level_values(0).unique():
        targets = piv.loc[r,(c, 'Targets')]
        if pd.isna(targets):
            annot_df.loc[r,c] = ''
        else:
            annot_df.loc[r, c] = f"{piv.loc[r, (c, 'Success Rate')]*100:,.1f}%<br>({piv.loc[r,(c, 'Successes')]:,.0f} / {targets:,.0f})"

# print(annot_df.head().to_string())
# print(sl.head().to_string())
# print(sl.index)

# Heatmap

cols = [col.replace(' ', '<br>') for col in sl.columns.get_level_values(0).drop('Targets')]
# cols = ['1st', '2nd', '3rd', '4th']

fig = heat_map(
    x=cols,
    y=sl.index,
    z=sl.values.tolist(),
    text=annot_df.values.tolist(),
    title=f'Receiving <span style="color: #006D2C">Success Rate</span> by {DOWN_COL}<br><sup>NFL Week {WEEK}: {AWAY_TEAM} @ {HOME_TEAM}</sup>',
    color_name='Success Rate'
)
fig.update_layout(
    height=900
)
fig.show()

# pio.write_image(fig, "Success Rate by Down & Distance - Colts vs. Rams.png", scale=6)


                           1st                      2nd & Short                      2nd & Medium                      2nd & Long                      3rd & Short                      3rd & Medium                      3rd & Long                      4th & Short                      4th & Medium                     
                     Successes Targets Success Rate   Successes Targets Success Rate    Successes Targets Success Rate  Successes Targets Success Rate   Successes Targets Success Rate    Successes Targets Success Rate  Successes Targets Success Rate   Successes Targets Success Rate    Successes Targets Success Rate
receiver_player_name                                                                                                                                                                                                                                                                                                        
A.Abdullah                 0.0     1.0     0.0000

## EPA

In [86]:
''' EPA by Play Type '''

epa_summary = game_data.groupby(['posteam', 'play_type']).aggregate({
    'posteam': 'size', 
    'ep': 'sum',
    'epa': 'sum'
}).rename(columns={'posteam': 'Plays'})
epa_summary['EPA / Play'] = epa_summary['epa'] / epa_summary['Plays']

print(epa_summary.groupby('posteam')['epa'].sum())
print(epa_summary.to_string())

posteam
IND    19.897320
TEN    -8.311419
Name: epa, dtype: float32
                     Plays         ep        epa  EPA / Play
posteam play_type                                           
IND     extra_point      5   4.657144   0.342857    0.068571
        field_goal       1   2.150450   0.849550    0.849550
        kickoff          3   4.185029   0.437633    0.145878
        no_play          1   2.011646  -1.522343   -1.522343
        pass            34  86.031448  11.726611    0.344900
        punt             4  -3.040758  -0.050828   -0.012707
        run             18  34.533447   8.113840    0.450769
TEN     extra_point      2   1.862857   0.137143    0.068571
        field_goal       1   1.626989  -1.949176   -1.949176
        kickoff          7   7.709158  -0.099621   -0.014232
        no_play          3   2.118842   3.248645    1.082882
        pass            41  75.061142  -8.368195   -0.204102
        punt             5  -2.842280  -3.875555   -0.775111
        run      

In [87]:
''' Drives EPA '''

epa_summary = game_data.groupby(['posteam', 'drive']).aggregate({
    'posteam': 'size', 
    'ep': 'first',
    'epa': 'sum'
}).rename(columns={'posteam': 'Plays', 'ep': 'Start EP'})
epa_summary['Drive End EP'] = epa_summary['Start EP'] + epa_summary['epa']
epa_summary['Drive Exp. Pts'] = np.where(epa_summary['Drive End EP'] < 0, 0, epa_summary['Drive End EP'])
epa_summary['EPA / Play'] = epa_summary['epa'] / epa_summary['Plays']

print(epa_summary.to_string())
print(epa_summary.groupby('posteam')['Drive Exp. Pts'].sum())

               Plays  Start EP       epa  Drive End EP  Drive Exp. Pts  EPA / Play
posteam drive                                                                     
IND     1.0       11  1.592993  1.407007      3.000000        3.000000    0.127910
        3.0        9  1.540722  5.527849      7.068571        7.068571    0.614205
        5.0        8  1.472162  5.596409      7.068572        7.068572    0.699551
        7.0        4  1.967553 -2.827820     -0.860267        0.000000   -0.706955
        9.0        5  0.542682 -0.781268     -0.238586        0.000000   -0.156254
        11.0       1  0.322187 -0.322187      0.000000        0.000000   -0.322187
        13.0       2  1.113361  5.955210      7.068572        7.068572    2.977605
        15.0      10  1.553515  5.515056      7.068572        7.068572    0.551506
        17.0       7  3.220508  3.848063      7.068571        7.068571    0.549723
        19.0       4  0.908393 -2.543533     -1.635140        0.000000   -0.635883
    

In [88]:
''' Best and Worst EPA Plays '''


for side in ['away', 'home']:
    col = 'home_team' if side == 'home' else 'away_team'
    team = game_data.loc[game_data.first_valid_index(), col]
    
    plays = game_data.loc[game_data['posteam_type'] == side, ['epa', 'qtr', 'time', 'yrdln', 'down', 'ydstogo', 'desc']].sort_values(by='epa', ascending=False)

    print(f'{team}')
    print('Best plays:')
    print(plays.head().to_string(index=False))
    print('Worst plays:')
    print(plays.tail().to_string(index=False))


TEN
Best plays:
     epa  qtr  time  yrdln  down  ydstogo                                                                                                                                                                                                                                 desc
4.529117  2.0 12:05 IND 42   3.0      6.0                                                                                                                                                  (12:05) (Shotgun) 2-T.Spears right end pushed ob at IND 1 for 41 yards (0-C.Bynum).
3.846205  4.0 02:40  IND 3   4.0      2.0                                                                                                                                                                                (2:40) 2-T.Spears left tackle for 3 yards, TOUCHDOWN.
3.186444  3.0 03:16 TEN 44   4.0      6.0 (3:16) (Shotgun) 1-C.Ward pass incomplete short middle to 11-V.Jefferson [97-L.Latu]. PENALTY on IND-23-K.Moore, Defensive Holdin